In [36]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize
import matplotlib.gridspec as gridspec
from datetime import date, timedelta
import geopandas as gpd

#import today date
date_today = date.today()
year_t,month_t,date_t=str(date_today).split('-')

# The SIR model differential equations.
def deriv(y, t, N, beta,gamma):
    S,I,R = y

    dSdt = -(beta*I/N)*S 
    dIdt = (beta*S/N)*I - gamma*I 
    dRdt = gamma*I 
    
    return dSdt, dIdt, dRdt

#Integration of the differential equations
    
def time_evo(N,beta,gamma,I0=1,R0=0,t=np.arange(0,365)):
    # Definition of the initial conditions
    # I0 and R0 denotes the number of initial infected people (I0) 
    # and the number of people that recovered and are immunized (R0)
    
    # t ise the timegrid
    
    S0=N-I0-R0  # number of people that can still contract the virus
    
    # Initial conditions vector
    y0 = S0, I0, R0

    # Integrate the SIR equations over the time grid, t.
    ret = odeint(deriv, y0, t, args=(N,beta,gamma))
    S, I, R = np.transpose(ret)
    
    return (t,S,I,R)

countries_list=['Albania',
                'Armenia',
                'Austria',
                'Azerbaijan',
                'Belarus',
                'Belgium',
                'Bosnia and Herzegovina',
                'Bulgaria',
                'Cyprus',
                'Croatia',
                'Czechia',
                'Denmark',
                'Estonia',
                'Finland',
                'France',
                'Georgia',
                'Germany',
                'Greece',
                'Hungary',
                'Iceland',
                'Ireland',
                'Israel',
                'Italy',
                'Kazakhstan',
                'Kyrgyzstan',
                'Latvia',
                'Lithuania',
                'Luxembourg',
                'Malta',
                'Moldova',
                'Monaco',
                'Montenegro',
                'Netherlands',
                'North Macedonia',
                'Norway',
                'Poland',
                'Portugal',
                'Romania',
                'Serbia',
                'Slovakia',
                'Slovenia',
                'Spain',
                'Sweden',
                'Switzerland',
                'Turkey',
                'Ukraine',
                'United Kingdom']

#IMPORT FILES WORLD
#i files sono: le righe sono le nazioni, le colonne i giorni del mese (DATE).

file_confirmed='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
file_deaths='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
file_recovered='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'

df_confirmed=pd.read_csv(file_confirmed)
df_deaths=pd.read_csv(file_deaths)
df_recovered=pd.read_csv(file_recovered)

countries_w_confirmed = df_confirmed['Country/Region']
countries_w_deaths = df_deaths['Country/Region']
countries_w_recovered = df_recovered['Country/Region']

#confirmed world
confirmed_world0 = df_confirmed.drop(['Province/State','Lat','Long'], 
                                    axis=1)
confirmed_world0.rename(index=countries_w_confirmed, inplace=True)
confirmed_world = confirmed_world0.drop(['Country/Region'], 
                                        axis=1).T.reset_index()
confirmed_world.rename(columns={'index':'Date'}, inplace=True)

#deaths world
deaths_world0 = df_deaths.drop(['Province/State','Lat','Long'], 
                                    axis=1)
deaths_world0.rename(index=countries_w_deaths, inplace=True)
deaths_world = deaths_world0.drop(['Country/Region'], 
                                        axis=1).T.reset_index()
deaths_world.rename(columns={'index':'Date'}, inplace=True)

#recovered world
recovered_world0 = df_recovered.drop(['Province/State','Lat','Long'], 
                                    axis=1)
recovered_world0.rename(index=countries_w_recovered, inplace=True)
recovered_world = recovered_world0.drop(['Country/Region'], 
                                        axis=1).T.reset_index()
recovered_world.rename(columns={'index':'Date'}, inplace=True)

confirmed_europe0 = confirmed_world[countries_list]
deaths_europe0 = deaths_world[countries_list]
recovered_europe0 = recovered_world[countries_list]

array_names=([])
for name in countries_list:
    array_names.append([name,list(countries_w_confirmed).count(name)])

Totale=pd.DataFrame()
for i in range(0, len(countries_list)):
    if array_names[i][1] > 1:
               Totale.insert(i, 
                              countries_list[i], 
                              value=confirmed_europe0[countries_list[i]].T.sum())
    elif array_names[i][1]==1:
                Totale.insert(i, 
                                countries_list[i], 
                                value=confirmed_europe0[countries_list[i]].T)

Totale.insert(0, 'Date', confirmed_world['Date'])

Deceduti=pd.DataFrame()
for i in range(0, len(countries_list)):
    if array_names[i][1] > 1:
            Deceduti.insert(i, 
                              countries_list[i], 
                              value=deaths_europe0[countries_list[i]].T.sum())
    elif array_names[i][1]==1:
            Deceduti.insert(i, 
                              countries_list[i], 
                              value=deaths_europe0[countries_list[i]].T)

Deceduti.insert(0, 'Date', deaths_world['Date'])

Guariti=pd.DataFrame()
for i in range(0, len(countries_list)):
    if array_names[i][1] > 1:
                Guariti.insert(i, 
                              countries_list[i], 
                              value=recovered_europe0[countries_list[i]].T.sum())
    elif array_names[i][1]==1:
                Guariti.insert(i, 
                                 countries_list[i], 
                                 value=recovered_europe0[countries_list[i]].T)

Guariti.insert(0, 'Date', recovered_world['Date'])

#Active Infected
Attualmente_positivi=pd.DataFrame()

for i in range(0, len(countries_list)):
    Attualmente_positivi.insert(i, 
                                  countries_list[i], 
                                  value=
                                     Totale[countries_list[i]]-
                                      Deceduti[countries_list[i]]-
                                      Guariti[countries_list[i]])

Attualmente_positivi.insert(0, 'Date', confirmed_world['Date'])

Totale.to_csv('output/10_tot_casi_europe_'+date_t+month_t+'.csv', index=True)
Deceduti.to_csv('output/10_deceduti_europe_'+date_t+month_t+'.csv', index=True)
Guariti.to_csv('output/10_guariti_europe_'+date_t+month_t+'.csv', index=True)
Attualmente_positivi.to_csv('output/10_attualmente_positivi_europe_'+date_t+month_t+'.csv', index=True)

#Daily variation infected
Variazione_giornaliera = pd.DataFrame(Attualmente_positivi['Date'].iloc[1:])

for name in countries_list:
    active_var=([])    
    for i in range(1,len(Attualmente_positivi)):
        active_var.append(Attualmente_positivi[name][i]-Attualmente_positivi[name][i-1])
    Variazione_giornaliera[name]=active_var

Variazione_giornaliera.to_csv('output/10_variazione_giornaliera_europe_'+date_t+month_t+'.csv', index=True)

def func_plot(df):
    
    y_world=[]
    n_cols=df.shape[1]
    
    for i in range(n_cols-4):
        y_world.append(df.iloc[:,i+4].sum())
    
    x_world2=df.columns[4:]
    x_world=pd.to_datetime(x_world2,infer_datetime_format=False)
    
    return (x_world,y_world)

#Generalization to other countries

def whichcountry(name):

    ######## INPUT PARAMETERS ########
    country=name
    t0=pd.to_datetime('2020-01-22')
    #################################

    mask_coun=df_confirmed['Country/Region']==country   # you can change the country here
    mask_coun_rec=df_recovered['Country/Region']==country

    df_confirmed_C=df_confirmed.loc[mask_coun,:]
    df_deaths_C=df_deaths.loc[mask_coun,:]
    df_recovered_C=df_recovered.loc[mask_coun_rec,:]

    ytot=np.array(func_plot(df_confirmed_C)[1])
    ydeaths=np.array(func_plot(df_deaths_C)[1])
    yrec=np.array(func_plot(df_recovered_C)[1])

    return ytot-ydeaths-yrec, ytot[-1], yrec[-1],ydeaths[-1]

xdata=pd.to_numeric(range(Attualmente_positivi.shape[0]))

today=len(xdata)

def minimizer(R0,t1=today-5,t2=today):
    array_country_bis=array_country
    
    #true data
    ydata_inf_2=array_country[t1:t2]
    xdata_2=np.arange(0,len(ydata_inf_2))
    
    #model
    fin_result=time_evo(60*10**6,1/14*R0,1/14,I0=ydata_inf_2[0])
    i_vec=fin_result[2]
    i_vec_2=i_vec[0:len(xdata_2)]
    
    #average error
    error=np.sum(np.abs(ydata_inf_2-i_vec_2)/ydata_inf_2)*100
    return error

minimizer_vec=np.vectorize(minimizer)

time_window=5

def minimizer_gen(t1,t2,xgrid=np.arange(0.1,5,0.01)):

    ygrid=minimizer_vec(xgrid,t1=t1,t2=t2)
    r0_ideal=round(xgrid[np.argmin(ygrid)],2)

    return r0_ideal

r0_today=[]
scangrid=np.linspace(0,3,400)
name_R0_array = []

for name in range(0, len(countries_list)):
    
    array_country=whichcountry(countries_list[name])[0]
    
    i = today-(time_window-1)
    min_today=minimizer_gen(i,i+time_window,scangrid)
    r0_today.append(min_today)
    #scangrid=np.linspace(0,5,200)
    name_R0_array.append([countries_list[name], min_today])

name_R0_df = pd.DataFrame(name_R0_array, columns=['Country', 'R0'])

countries_hist=['United Kingdom',
                'Ukraine',
                'Poland',
                'Greece',
                'Netherlands',
                'Portugal',
                'Belgium',
                'France',
                'Slovenia',
                'Serbia',
                'Spain',
                'Italy',
                'Sweden',
                'Austria',
                'Slovakia',
                'Turkey']

hist_list=[]
for i in range(len(countries_hist)):
    ind = name_R0_df.loc[name_R0_df['Country'] == countries_hist[i]].index[0]
    hist_list.append([name_R0_df['Country'][ind], name_R0_df['R0'][ind]])
hist_df = pd.DataFrame(hist_list, columns=['Country', 'R0'])
hist_df.to_csv('output/10_R0_europe_hist_'+date_t+month_t+'.csv')

##import yesterday date
#yesterday = date.today() - timedelta(days=1)
#year_y,month_y,date_y=str(yesterday).split('-')
#
#r0_countries_imp = pd.read_excel('input/input.xlsx')
#
#
#row_today=pd.DataFrame(np.reshape(r0_today,(1, len(countries_list))),
#                       index= [str(yesterday)],
#                       columns=countries_list).reset_index()
#row_today.rename(columns={'index':'Data'}, inplace=True)
#row_today.index = [len(r0_countries_imp)]
#
#export_today = pd.concat([r0_imp_noindex,row_today])
#export_today.to_excel('output/10_R0_europe_curve_'+date_t+month_t+'.xlsx',index=True)
#export_today.to_excel('input/input.xlsx', index = True)
#
#r0_to_join = pd.Series(name_R0_df['R0'])
#r0_to_join.index = name_R0_df['Country']
#confirmed_to_join = Totale.iloc[-1, 1:]
#deaths_to_join = Deceduti.iloc[-1, 1:]
#recovered_to_join = Guariti.iloc[-1, 1:]
#ai_to_join = Attualmente_positivi.iloc[-1, 1:]
#
#frame = {'R0':r0_to_join,
#         'Confirmed': confirmed_to_join, 
#         'Deaths': deaths_to_join, 
#         'Recovered':recovered_to_join, 
#         'Active Infected': ai_to_join}
#
#df_to_join = pd.DataFrame(frame)
#df_to_join.rename(index={'Czechia':'Czech Republic', 
#                         'Moldova':'Republic of Moldova', 
#                         'North Macedonia':'The former Yugoslav Republic of Macedonia'})
#df_to_join.reset_index()
#
##Map
#map = gpd.read_file("https://raw.githubusercontent.com/leakyMirror/map-of-europe/master/GeoJSON/europe.geojson")
#map = map.join(df_to_join, on='NAME', how='left')

In [37]:
#import yesterday date
yesterday = date.today() - timedelta(days=1)
year_y,month_y,date_y=str(yesterday).split('-')

r0_countries_imp = pd.read_excel('input/input.xlsx')


In [38]:
r0_countries_imp.tail()

,Date,Albania,Armenia,Austria,Azerbaijan,Belarus,Belgium,Bosnia and Herzegovina,Bulgaria,Cyprus,...,Romania,Serbia,Slovakia,Slovenia,Spain,Sweden,Switzerland,Turkey,Ukraine,United Kingdom
98,2020-06-17,2.75,1.46,1.23,1.50,0.89,1.03,1.84,1.25,0.91,...,1.39,1.21,0.63,2.20,1.07,1.23,0.20,1.23,1.41,1.07
99,2020-06-18,2.49,1.30,1.53,1.57,0.89,1.03,2.10,1.35,0.77,...,1.28,1.19,0.34,2.21,1.07,1.28,0.37,1.25,1.32,1.07
100,2020-06-19,2.33,1.14,1.44,1.52,0.68,1.03,1.61,1.47,0.64,...,1.33,1.31,0.00,2.64,1.04,1.26,1.89,1.10,1.27,1.07
101,2020-06-20,1.84,0.99,1.26,1.67,0.54,1.03,1.56,1.40,0.85,...,1.40,1.62,1.13,2.73,1.03,1.23,1.97,0.95,1.26,1.07
102,2020-06-21,1.82,0.84,1.42,1.68,0.58,1.03,1.38,1.35,0.79,...,1.25,1.66,1.54,1.83,1.00,1.13,1.54,0.90,1.29,1.07


In [39]:
row_today=pd.DataFrame(np.reshape(r0_today,(1, len(countries_list))),
                       index= [str(yesterday)],
                       columns=countries_list).reset_index()
row_today.rename(columns={'index':'Date'}, inplace=True)
row_today.index = [len(r0_countries_imp)]

In [40]:
row_today.head()

,Date,Albania,Armenia,Austria,Azerbaijan,Belarus,Belgium,Bosnia and Herzegovina,Bulgaria,Cyprus,...,Romania,Serbia,Slovakia,Slovenia,Spain,Sweden,Switzerland,Turkey,Ukraine,United Kingdom
103,2020-06-21,1.6,0.8,1.21,1.68,0.6,1.02,1.44,1.26,0.83,...,1.26,1.71,1.97,0.05,0.95,1.0,0.89,0.87,1.3,1.07


In [41]:
export_today = pd.concat([r0_countries_imp,row_today])

In [42]:
export_today.tail(10)

,Date,Albania,Armenia,Austria,Azerbaijan,Belarus,Belgium,Bosnia and Herzegovina,Bulgaria,Cyprus,...,Romania,Serbia,Slovakia,Slovenia,Spain,Sweden,Switzerland,Turkey,Ukraine,United Kingdom
94,2020-06-13 00:00:00,1.85,1.20,0.66,1.62,0.80,1.03,1.68,1.78,1.19,...,1.11,1.62,1.19,1.94,1.10,1.40,0.61,0.69,1.21,1.07
95,2020-06-14 00:00:00,1.82,1.56,0.63,1.50,0.79,1.03,1.48,1.77,1.21,...,1.17,1.96,1.28,1.85,1.10,1.42,0.76,1.09,1.28,1.07
96,2020-06-15 00:00:00,2.09,1.55,0.62,1.50,0.76,1.03,1.50,1.56,1.25,...,1.30,1.50,1.26,2.27,1.07,1.32,1.79,1.13,1.36,1.07
97,2020-06-16 00:00:00,2.71,1.45,0.85,1.50,0.90,1.03,1.67,1.38,1.01,...,1.27,1.34,1.26,2.19,1.07,1.27,0.72,1.26,1.37,1.07
98,2020-06-17 00:00:00,2.75,1.46,1.23,1.50,0.89,1.03,1.84,1.25,0.91,...,1.39,1.21,0.63,2.20,1.07,1.23,0.20,1.23,1.41,1.07
99,2020-06-18 00:00:00,2.49,1.30,1.53,1.57,0.89,1.03,2.10,1.35,0.77,...,1.28,1.19,0.34,2.21,1.07,1.28,0.37,1.25,1.32,1.07
100,2020-06-19 00:00:00,2.33,1.14,1.44,1.52,0.68,1.03,1.61,1.47,0.64,...,1.33,1.31,0.00,2.64,1.04,1.26,1.89,1.10,1.27,1.07
101,2020-06-20 00:00:00,1.84,0.99,1.26,1.67,0.54,1.03,1.56,1.40,0.85,...,1.40,1.62,1.13,2.73,1.03,1.23,1.97,0.95,1.26,1.07
102,2020-06-21 00:00:00,1.82,0.84,1.42,1.68,0.58,1.03,1.38,1.35,0.79,...,1.25,1.66,1.54,1.83,1.00,1.13,1.54,0.90,1.29,1.07
103,2020-06-21,1.60,0.80,1.21,1.68,0.60,1.02,1.44,1.26,0.83,...,1.26,1.71,1.97,0.05,0.95,1.00,0.89,0.87,1.30,1.07


In [43]:
export_today.to_excel('output/10_R0_europe_curve_'+date_t+month_t+'.xlsx',index=True)
export_today.to_excel('input/input.xlsx', index = True)

In [44]:
r0_to_join = pd.Series(name_R0_df['R0'])
r0_to_join.index = name_R0_df['Country']

In [45]:
r0_to_join.head()

Country
Albania       1.60
Armenia       0.80
Austria       1.21
Azerbaijan    1.68
Belarus       0.60
Name: R0, dtype: float64

In [46]:
confirmed_to_join = Totale.iloc[-1, 1:]
deaths_to_join = Deceduti.iloc[-1, 1:]
recovered_to_join = Guariti.iloc[-1, 1:]
ai_to_join = Attualmente_positivi.iloc[-1, 1:]

frame = {'R0':r0_to_join,
         'Confirmed': confirmed_to_join, 
         'Deaths': deaths_to_join, 
         'Recovered':recovered_to_join, 
         'Active Infected': ai_to_join}

In [47]:
df_to_join = pd.DataFrame(frame)
df_to_join.rename(index={'Czechia':'Czech Republic', 
                         'Moldova':'Republic of Moldova', 
                         'North Macedonia':'The former Yugoslav Republic of Macedonia'})

,R0,Confirmed,Deaths,Recovered,Active Infected
Country,,,,,
Albania,1.60,1962,44,1134,784
Armenia,0.80,20268,350,9002,10916
Austria,1.21,17341,690,16197,454
Azerbaijan,1.68,12729,154,6799,5776
Belarus,0.60,58505,346,37666,20493
Belgium,1.02,60550,9696,16771,34083
Bosnia and Herzegovina,1.44,3273,169,2241,863
Bulgaria,1.26,3905,199,2074,1632
Cyprus,0.83,986,19,824,143


In [48]:
df_to_join.reset_index()

,Country,R0,Confirmed,Deaths,Recovered,Active Infected
0,Albania,1.60,1962,44,1134,784
1,Armenia,0.80,20268,350,9002,10916
2,Austria,1.21,17341,690,16197,454
3,Azerbaijan,1.68,12729,154,6799,5776
4,Belarus,0.60,58505,346,37666,20493
5,Belgium,1.02,60550,9696,16771,34083
6,Bosnia and Herzegovina,1.44,3273,169,2241,863
7,Bulgaria,1.26,3905,199,2074,1632
8,Cyprus,0.83,986,19,824,143
9,Croatia,3.00,2317,107,2142,68


In [49]:
#Map
map = gpd.read_file("https://raw.githubusercontent.com/leakyMirror/map-of-europe/master/GeoJSON/europe.geojson")
map = map.join(df_to_join, on='NAME', how='left')

In [50]:
map


,FID,FIPS,ISO2,ISO3,UN,NAME,AREA,POP2005,REGION,SUBREGION,LON,LAT,geometry,R0,Confirmed,Deaths,Recovered,Active Infected
0,0.0,AJ,AZ,AZE,31,Azerbaijan,8260,8352021,142,145,47.395,40.430,"(POLYGON ((45.083321 39.76804400000015, 45.266...",1.68,12729,154,6799,5776
1,0.0,AL,AL,ALB,8,Albania,2740,3153731,150,39,20.068,41.143,"POLYGON ((19.436214 41.021065, 19.450554 41.05...",1.60,1962,44,1134,784
2,0.0,AM,AM,ARM,51,Armenia,2820,3017661,142,145,44.563,40.534,"(POLYGON ((45.57305100000013 40.632488, 45.528...",0.80,20268,350,9002,10916
3,0.0,BK,BA,BIH,70,Bosnia and Herzegovina,5120,3915238,150,39,17.786,44.169,"POLYGON ((17.649841 42.889076, 17.578526 42.94...",1.44,3273,169,2241,863
4,0.0,BU,BG,BGR,100,Bulgaria,11063,7744591,150,151,25.231,42.761,"POLYGON ((27.879166 42.841103, 27.894997 42.80...",1.26,3905,199,2074,1632
5,0.0,CY,CY,CYP,196,Cyprus,924,836321,142,145,33.219,35.043,"POLYGON ((33.652618 35.354103, 33.713051 35.38...",0.83,986,19,824,143
6,0.0,DA,DK,DNK,208,Denmark,4243,5416945,150,154,9.264,56.058,"(POLYGON ((11.513887 54.82972000000012, 11.564...",1.07,12591,600,11482,509
7,0.0,EI,IE,IRL,372,Ireland,6889,4143294,150,154,-8.152,53.177,"(POLYGON ((-9.65639 53.222221, -9.663334000000...",1.11,25379,1715,22698,966
8,0.0,EN,EE,EST,233,Estonia,4239,1344312,150,154,25.793,58.674,"(POLYGON ((23.990829 58.099998, 23.97805000000...",0.64,1981,69,1764,148
9,0.0,AU,AT,AUT,40,Austria,8245,8291979,150,155,14.912,47.683,"POLYGON ((13.833611 48.773605, 13.858055 48.77...",1.21,17341,690,16197,454


In [51]:
map.to_file('output/10_mappa_R0_europa_'+date_t+month_t+'.geojson', driver='GeoJSON')

GeometryTypeValidationError: Record's geometry type does not match collection schema's geometry type: 'MultiPolygon' != 'Polygon'